In [ ]:
# 0️⃣ Install & Import Libraries
!pip install pulp kaggle
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value

# 1️⃣ (Optional) Download dataset via Kaggle CLI
# kaggle datasets download datasetengineer/logistics-and-supply-chain-dataset
# unzip and update the path below.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 95.7 MB/s eta 0:00:00


In [ ]:

# 2️⃣ Load CSV
df = pd.read_csv("/content/dynamic_supply_chain_logistics_dataset.csv")

# Inspect columns
print(df.columns)
print(df.head())



Index(['timestamp', 'vehicle_gps_latitude', 'vehicle_gps_longitude',
       'fuel_consumption_rate', 'eta_variation_hours',
       'traffic_congestion_level', 'warehouse_inventory_level',
       'loading_unloading_time', 'handling_equipment_availability',
       'order_fulfillment_status', 'weather_condition_severity',
       'port_congestion_level', 'shipping_costs', 'supplier_reliability_score',
       'lead_time_days', 'historical_demand', 'iot_temperature',
       'cargo_condition_status', 'route_risk_level', 'customs_clearance_time',
       'driver_behavior_score', 'fatigue_monitoring_score',
       'disruption_likelihood_score', 'delay_probability',
       'risk_classification', 'delivery_time_deviation'],
      dtype='object')
             timestamp  vehicle_gps_latitude  vehicle_gps_longitude  \
0  2021-01-01 00:00:00             40.375568             -77.014318   
1  2021-01-01 01:00:00             33.507818            -117.036902   
2  2021-01-01 02:00:00             30.02064

In [ ]:
# 3. Preprocess and filter usable routes
df = df.dropna(subset=['shipping_costs', 'traffic_congestion_level', 'supplier_reliability_score',
                       'delay_probability', 'warehouse_inventory_level', 'disruption_likelihood_score'])

# Filter routes with low disruption and good inventory
df = df[(df['warehouse_inventory_level'] > 50) & (df['disruption_likelihood_score'] < 0.6)].reset_index()




In [ ]:
# 4. Define the optimization problem
model = LpProblem("Optimal_Delivery_Route_Selection", LpMinimize)

In [ ]:
# Number of routes to select (can be adjusted)
ROUTE_LIMIT = 10


In [ ]:
x = LpVariable.dicts("SelectRoute", df.index, cat='Binary')

In [ ]:
# 6. Define weights and penalties
traffic_penalty = 5
delay_penalty = 10
reliability_benefit = 15

In [ ]:

# 7. Objective: Minimize cost + traffic + delay - reliability
model += lpSum([
    (df.loc[i, 'shipping_costs'] +
     df.loc[i, 'traffic_congestion_level'] * traffic_penalty +
     df.loc[i, 'delay_probability'] * delay_penalty -
     df.loc[i, 'supplier_reliability_score'] * reliability_benefit) * x[i]
    for i in df.index
]), "Total_Weighted_Cost"


In [ ]:
model += lpSum([x[i] for i in df.index]) == ROUTE_LIMIT, "Route_Limit"

In [ ]:
# 9. Solve the model
model.solve()

1

In [ ]:
print("🚛 Selected Optimal Routes:")
for i in df.index:
    if x[i].value() == 1:
        row = df.loc[i]
        print(f"- Route {i}: Cost ₹{row['shipping_costs']}, Traffic {row['traffic_congestion_level']}, Delay Prob {row['delay_probability']}, Reliability {row['supplier_reliability_score']}")


🚛 Selected Optimal Routes:
- Route 2122: Cost ₹100.01451270025788, Traffic 0.0002348350828762, Delay Prob 0.7717070945715082, Reliability 0.9996981361142818
- Route 2315: Cost ₹103.90654229897734, Traffic 0.0988205437016916, Delay Prob 0.0330787811677864, Reliability 0.99305708701213
- Route 2761: Cost ₹100.13931816414372, Traffic 0.0019782935860833, Delay Prob 0.1073536332509015, Reliability 0.5966393734687859
- Route 2898: Cost ₹100.38661074981108, Traffic 0.0037791385373399, Delay Prob 0.1009031762143384, Reliability 0.494648401923942
- Route 3150: Cost ₹100.1732145654583, Traffic 0.4221329263781292, Delay Prob 0.3217559420157662, Reliability 0.7688121775462846
- Route 3337: Cost ₹100.19019196732862, Traffic 1.264316255208446, Delay Prob 0.0884461986280809, Reliability 0.9078171455403106
- Route 3746: Cost ₹100.13935100985265, Traffic 0.2113706037641032, Delay Prob 0.3157713662656101, Reliability 0.9997324777846848
- Route 4082: Cost ₹100.295596232285, Traffic 0.4210524085886721, De

In [ ]:
import streamlit as st
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value

st.set_page_config(page_title="📦 Logistics Optimization Dashboard", layout="wide")

st.title("📦 Real-Time Supply Chain Optimization")

# Upload CSV
uploaded_file = st.file_uploader("Upload your logistics CSV file", type=["csv"])
if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.success("File uploaded successfully!")
    st.write("📊 Preview of dataset:", df.head())

    # Constraint sliders
    route_limit = st.slider("🚛 Number of Routes to Select", min_value=1, max_value=50, value=10)
    inventory_threshold = st.slider("🏭 Min Warehouse Inventory Level", 0, 200, 50)
    disruption_threshold = st.slider("⚠️ Max Disruption Likelihood Score", 0.0, 1.0, 0.6)

    # Weights
    traffic_penalty = st.slider("🚦 Traffic Penalty Multiplier", 1, 20, 5)
    delay_penalty = st.slider("⏱️ Delay Penalty Multiplier", 1, 20, 10)
    reliability_benefit = st.slider("✅ Reliability Benefit Multiplier", 1, 30, 15)

    # Filter and preprocess
    required_cols = ['shipping_costs', 'traffic_congestion_level', 'supplier_reliability_score',
                     'delay_probability', 'warehouse_inventory_level', 'disruption_likelihood_score']

    if all(col in df.columns for col in required_cols):
        df = df.dropna(subset=required_cols)
        df = df[(df['warehouse_inventory_level'] > inventory_threshold) &
                (df['disruption_likelihood_score'] < disruption_threshold)].reset_index(drop=True)

        st.markdown(f"✅ **Filtered to {len(df)} viable routes.**")

        # Optimization
        model = LpProblem("Logistics_Optimization", LpMinimize)
        x = LpVariable.dicts("SelectRoute", df.index, cat='Binary')

        model += lpSum([
            (df.loc[i, 'shipping_costs'] +
             df.loc[i, 'traffic_congestion_level'] * traffic_penalty +
             df.loc[i, 'delay_probability'] * delay_penalty -
             df.loc[i, 'supplier_reliability_score'] * reliability_benefit) * x[i]
            for i in df.index
        ]), "Total_Weighted_Cost"

        model += lpSum([x[i] for i in df.index]) == route_limit, "Route_Limit"

        if st.button("🚀 Optimize Routes"):
            model.solve()
            selected = [i for i in df.index if x[i].value() == 1]
            selected_df = df.loc[selected]

            st.subheader("✅ Optimized Route Selection")
            st.write(selected_df)

            st.metric("💰 Total Minimized Cost", f"₹{value(model.objective):.2f}")
            st.download_button("📥 Download Optimized Routes as CSV",
                               selected_df.to_csv(index=False), file_name="optimized_routes.csv")
    else:
        st.error("❌ Dataset is missing required columns.")


2025-06-22 14:41:30.921 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 14:41:30.927 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 14:41:31.105 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-22 14:41:31.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 14:41:31.116 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 14:41:31.122 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 14:41:31.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [ ]:
import gradio as gr
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value
import tempfile

def optimize_routes(file, route_limit, inventory_threshold, disruption_threshold,
                    traffic_penalty, delay_penalty, reliability_benefit):

    df = pd.read_csv(file.name)

    required_cols = ['shipping_costs', 'traffic_congestion_level', 'supplier_reliability_score',
                     'delay_probability', 'warehouse_inventory_level', 'disruption_likelihood_score']

    if not all(col in df.columns for col in required_cols):
        return "❌ Dataset is missing required columns.", None

    # Filter rows
    df = df.dropna(subset=required_cols)
    df = df[(df['warehouse_inventory_level'] > inventory_threshold) &
            (df['disruption_likelihood_score'] < disruption_threshold)]

    if len(df) == 0:
        return "❌ No valid routes after filtering. Adjust your constraints.", None

    # Limit to top 500 lowest shipping cost routes
    df = df.nsmallest(500, 'shipping_costs').reset_index(drop=True)

    if len(df) < route_limit:
        return f"❌ Only {len(df)} valid routes available. Lower the route limit.", None

    # Setup LP Model
    model = LpProblem("Fast_Logistics_Optimization", LpMinimize)
    x = LpVariable.dicts("SelectRoute", df.index, lowBound=0, upBound=1, cat='Continuous')  # continuous approx

    model += lpSum([
        (df.loc[i, 'shipping_costs'] +
         df.loc[i, 'traffic_congestion_level'] * traffic_penalty +
         df.loc[i, 'delay_probability'] * delay_penalty -
         df.loc[i, 'supplier_reliability_score'] * reliability_benefit) * x[i]
        for i in df.index
    ])

    model += lpSum([x[i] for i in df.index]) == route_limit, "Route_Limit"

    # Solve
    model.solve()

    # Select top x[i] values (approx 1.0 means chosen)
    selected = sorted(df.index, key=lambda i: -x[i].value())[:route_limit]
    selected_df = df.loc[selected]

    # Save results
    temp_csv = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
    selected_df.to_csv(temp_csv.name, index=False)

    return selected_df, temp_csv.name

# Gradio UI
iface = gr.Interface(
    fn=optimize_routes,
    inputs=[
        gr.File(label="Upload Logistics CSV"),
        gr.Slider(1, 50, value=10, label="🚚 Number of Routes to Select"),
        gr.Slider(0, 200, value=50, label="🏭 Min Warehouse Inventory"),
        gr.Slider(0.0, 1.0, value=0.6, label="⚠️ Max Disruption Score"),
        gr.Slider(1, 20, value=5, label="🚦 Traffic Penalty"),
        gr.Slider(1, 20, value=10, label="⏱️ Delay Penalty"),
        gr.Slider(1, 30, value=15, label="✅ Reliability Benefit"),
    ],
    outputs=[
        gr.Dataframe(label="✅ Optimized Routes"),
        gr.File(label="📥 Download Selected Routes CSV")
    ],
    title="📦 Fast Logistics Optimization Dashboard",
    description="Upload a large logistics dataset and get optimized shipping routes in seconds.",
    theme="soft"
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ae8799a48dcccc4eae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
